In [7]:
import spacy
nlp = spacy.load('en')

In [65]:
s1 = "The role of Gillian B. Loeb was played by Colin McFarlane in 2013."
s2 = "Colin McFarlane reprises his role as Gillian B. Loeb, the Police Commissioner of Gotham."
s3 = "The role of Gillian B. Loeb was played by Colin McFarlane in 2013."
s4 = "Sebastian Stan reprising his role as Bucky Barnes."
s5 = "Terrence Howard was announced in the role of Jim Rhodes in October 2006."
s6 = "Sebastian Stan makes an uncredited appearance in the post-credits scene reprising his role as Bucky Barnes."
s7 = "the role of Sonny Corleone was awarded to Carmine Caridi."
s8 = "Robert Downey played role of IronMan."

In [116]:
sentence = s8
parse(sentence)

Matching with Played role template for : 'Robert Downey played role of IronMan.'
--------- Extracted Information -----------
Actor name :  Robert Downey
character name :  IronMan
Time : <no-match>


In [95]:
def parse(statement1):
    isValidForPlayedRoleTemplate = checkPlayedRoleTemplateEligibility(statement1)
    if(isValidForPlayedRoleTemplate):
        #print("is Valid? : " , isValidForPlayedRoleTemplate)
        print("--------- Extracted Information -----------")
        
        actorName = extractActorName(statement1)
        print("Actor name : " , actorName)

        characterName = extractCharacterName(statement1)
        print("character name : " , characterName)

        year = extractYear(statement1)
        print("Time :" , year)
    else:
        print("NO - PARSE")

In [11]:
# statement1 = "Colin McFarlane reprises his role as Gillian B. Loeb, the Police Commissioner of Gotham."
statement1 = "The role of Gillian B. Loeb was played by Colin McFarlane in 2013."
# statement1 = "Sebastian Stan reprising his role as Bucky Barnes."
# statement1 = "Terrence Howard was announced in the role of Jim Rhodes in October 2006."
# statement1 = "Sebastian Stan makes an uncredited appearance in the post-credits scene reprising his role as Bucky Barnes."
# statement1 = "the role of Sonny Corleone was awarded to Carmine Caridi."

isValidForPlayedRoleTemplate = checkPlayedRoleTemplateEligibility(statement1)
if(isValidForPlayedRoleTemplate):
    print("is Valid? : " , isValidForPlayedRoleTemplate)
    actorName = extractActorName(statement1)
    print("Extracted Actor name : " , actorName)

    characterName = extractCharacterName(statement1)
    print("Extracted character name : " , characterName)

    year = extractYear(statement1)
    print("extracted year is:" , year)

NameError: name 'checkPlayedRoleTemplateEligibility' is not defined

In [96]:
def extractActorName(statement):
    targetVerbLemmas = ['play', 'reprise', 'announce', 'award']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    
    #print(verbText, verbTextDNode)
    actorName = "<uninitialized>"
    
    actorName = getActor(statement, verbText)
    
    return actorName

In [97]:
def extractCharacterName(statement):
    targetVerbLemmas = ['play', 'reprise', 'announce', 'award']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    characterName = "<uninitialized>"
    
#     handle when release is with premiered.
    '''
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        print("verbTextChanged to : "+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    '''

    #print("vt:"+verbText)

    characterName = getCharacter(statement,verbText)
    
    return characterName

In [98]:
def extractYear(statement):
    doc = nlp(statement)
    for ent in doc.ents:
        if (ent.label_ == "DATE" or ent.label_ == "TIME"):
            return ent.text
    
    return "<no-match>"

In [99]:
def getActor(statement, verbText):
    
    doc = nlp(statement)
    
    for token in doc:
        if (token.dep_ == "nsubj" and token.head.text == verbText):
            #print("token nsubj : " + token.text)
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        
        elif (token.dep_ == "nsubjpass" and token.head.lemma_ == "announce"):
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        
        elif (token.head.text == verbText or token.text == verbText):
            agent = extractOwnerEntitySyntactically(statement)
            return agent
        

In [100]:
def getCharacter(statement, verbText):
    doc = nlp(statement)
    theme = "<uninitialized>" #movie
    
    
    if (check_word_pattern(statement, " role of ")):
        for token in doc:
            if (token.head.text == "of"):
                if (token.head.head.text == "role"):
                    theme = getNounChunkThatContainsNoun(statement, token)
                    return theme
                
    else:
        for token in doc:
            if ((token.text == "as" and token.head.text == verbText) or (token.text == "as" and token.head.text == "role")):
                for child in token.children:
                    theme = getNounChunkThatContainsNoun(statement, child)
                    return theme
    '''
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (check_word_pattern(statement, " role of ")):
            print("token nsubjpass : " + token.text)
            
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif (token.dep_ == "dobj" and token.head.text == verbText):
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif ((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            print("token conj : " + token.text)
            if (token.head.dep_ == "ROOT"):
                print("token.head: " + token.head.text)
                token1 = token.head
                return getMovie(statement, token1.text)
                
                
                if (token.dep_ == "nsubjpass" and token.head.text == token1.text):
                    theme = getNounChunkThatContainsNoun(statement, token)
                    return theme
                '''
    
    return "<no-match>"

In [101]:
def extractOwnerEntitySyntactically(statement):
    doc = nlp(statement)
    verbTokens = filterVerbTokens(statement)
    ORGEntity = []
    
    for token in verbTokens:
        if isExpectedVerbToken(token):
            #print("Expected token is:"+token.text)
            #expected children [was, theatrically, in, on, by]
            byToken = None
            for child in token.children: 
                #usually release location starts with "in"
                if child.text == "by" or child.text == "to":
                    byToken = child
            #print("bytoken: " , byToken) 
            
            if byToken == None:
                #means no direct by available so lets look at some conj child
                for child in token.children: 
                    #usually release location starts with "in"
                    if child.dep_ == "conj":
                        for grandChild in child.children:
                            if grandChild.text == "by":
                                byToken = grandChild
            
            if byToken != None:
                for subChild in byToken.children:
                    if(getORGorPersonThatContainsToken(statement, subChild) != "<no-match>"):
                        ORGEntity.append(getORGorPersonThatContainsToken(statement, subChild))
                    else:
                        ORGEntity.append(getNounChunkThatContainsNoun(statement, subChild))
                    #print("subchild : " + subChild.text)
                    if (subChild.dep_ == "pobj" and subChild.head.text == "by"):
                        grandChildren = subChild.children
                        childEntity = getORGEntityForGrandChild1(statement, grandChildren, ORGEntity)
                        #print("child entity:    " , str(childEntity))
                        return list(set(childEntity))
                        #ORGEntity.clear()
                        #for item in childEntity:
                            #ORGEntity.append(item)
                    return ORGEntity
    return "<no-match>"

In [102]:
def getORGEntityForGrandChild1(statement, children, ORGEntity):
    '''
    New definition
    '''
    doc = nlp(statement)
    
    for child in children:
        sizeChild = sum(1 for item in child.children)
        #print(child, sizeChild)
        if sizeChild > 0:
            ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
            #print(ORGEntity)
            #ORGEntity.append(getORGEntityForGrandChild1(statement, child.children, ORGEntity))
            getORGEntityForGrandChild1(statement, child.children, ORGEntity)
        if sizeChild == 0:
            if (getORGorPersonThatContainsToken(statement, child) != "<no-match>"):
                ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
    
    return ORGEntity

In [103]:
def getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas):
    """
    this will reutrn exact apperance of verb in statement.
    ex: statement = "Movie was written by James.", targetVerbLemmas = ['write','script']
    this will return "written" as it's lemma matches with one of the targetVerbLemmas
    """
    verbTokens = filterVerbTokens(statement)
    for token in verbTokens:
        if token.lemma_ in targetVerbLemmas:
            return token.text
    return "<no-match>"

In [104]:
def isExpectedVerbToken(token):
    return ((token.lemma_ in ["play"]) or (token.lemma_ in ["reprise"]) or (token.lemma_ in ["announce"]) or (token.lemma_ in ["award"]))

In [105]:
def getORGorPersonThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return "<no-match>"

In [106]:
import re
def calculateStartPositionOfToken(statement, targetToken):
    """
    this should return start position of the token
    """
    matches = [m.start() for m in re.finditer(targetToken.text, statement)]
    if(len(matches)==1):
        return matches[0]
    

    huristicPosition = 0
    doc = nlp(statement)
    
    i = 0
    for token in doc:
        if i < targetToken.i:
            huristicPosition = huristicPosition + len(token.text)+1
            i = i+1
        else:
            break
        
    distances = []
    for match in matches:
        distances.append(abs(match-huristicPosition))
        
    minIndex = distances.index(min(distances))
    
    return matches[minIndex]

In [107]:
def getNounChunkThatContainsNoun(statement, nounSubject):
    doc = nlp(statement)
    for chunk in doc.noun_chunks:
        if nounSubject.text in chunk.text:
            return chunk.text
    return "<no-match>"

In [108]:
def checkPlayedRoleTemplateEligibility(statement):
    print("Matching with Played role template for : '" + statement + "'")
    hasValidVerb = check_verb_match(statement)
    containsRole = check_role_match(statement)
    #print("Has valid verb:" + str(hasValidVerb))
    #print("Has valid role:" + str(containsRole))
    
    if (hasValidVerb and containsRole):
        return True
    else:
        return False

In [109]:
def check_verb_match(statement):
    verbTokens = filterVerbTokens(statement)
    
    #print(verbTokens)
    
    targetVerbs = ['play', 'reprise', 'announce', 'award']
    for token in verbTokens:
        if token.lemma_ in targetVerbs:
            return True
    return False

In [110]:
def filterVerbTokens(statement):
    '''
    return exact same words appeared in the statement
    '''
    doc = nlp(statement)
    result = list(filter(lambda token: (token.pos_ == "VERB" and token.lemma_ != "be"), doc))
    return result

In [111]:
filterVerbTokens('Colin McFarlane reprised his role as Gillian B. Loeb, the Police Commissioner of Gotham.')

[reprised]

In [112]:
def check_word_pattern(statement, pattern1):
    return ((pattern1 in statement))

In [113]:
def check_role_match(statement):
    return ((" role " in statement) or ("Role " in statement))

In [114]:
def get_dependency_tree_nodes(sentence):
    nodes = []
    doc = nlp(sentence)
    for token in doc:
        nodes.append([token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children]])
    return nodes

In [115]:
get_dependency_tree_nodes('Colin McFarlane reprises his role as Gillian B. Loeb, the Police Commissioner of Gotham.')

[['Colin', 'compound', 'McFarlane', 'PROPN', []],
 ['McFarlane', 'nsubj', 'reprises', 'VERB', [Colin]],
 ['reprises', 'ROOT', 'reprises', 'VERB', [McFarlane, role, as, .]],
 ['his', 'poss', 'role', 'NOUN', []],
 ['role', 'dobj', 'reprises', 'VERB', [his]],
 ['as', 'prep', 'reprises', 'VERB', [Loeb]],
 ['Gillian', 'compound', 'Loeb', 'PROPN', []],
 ['B.', 'compound', 'Loeb', 'PROPN', []],
 ['Loeb', 'pobj', 'as', 'ADP', [Gillian, B., ,, Commissioner]],
 [',', 'punct', 'Loeb', 'PROPN', []],
 ['the', 'det', 'Commissioner', 'PROPN', []],
 ['Police', 'compound', 'Commissioner', 'PROPN', []],
 ['Commissioner', 'appos', 'Loeb', 'PROPN', [the, Police, of]],
 ['of', 'prep', 'Commissioner', 'PROPN', [Gotham]],
 ['Gotham', 'pobj', 'of', 'ADP', []],
 ['.', 'punct', 'reprises', 'VERB', []]]

In [89]:
get_dependency_tree_nodes('The role of Gillian B. Loeb was played by Colin McFarlane.')

[['The', 'det', 'role', 'NOUN', []],
 ['role', 'nsubjpass', 'played', 'VERB', [The, of]],
 ['of', 'prep', 'role', 'NOUN', [Loeb]],
 ['Gillian', 'compound', 'Loeb', 'PROPN', []],
 ['B.', 'compound', 'Loeb', 'PROPN', []],
 ['Loeb', 'pobj', 'of', 'ADP', [Gillian, B.]],
 ['was', 'auxpass', 'played', 'VERB', []],
 ['played', 'ROOT', 'played', 'VERB', [role, was, by, .]],
 ['by', 'agent', 'played', 'VERB', [McFarlane]],
 ['Colin', 'compound', 'McFarlane', 'PROPN', []],
 ['McFarlane', 'pobj', 'by', 'ADP', [Colin]],
 ['.', 'punct', 'played', 'VERB', []]]

In [90]:
get_dependency_tree_nodes('Terrence Howard was announced in the role of Jim Rhodes in October 2006.')

[['Terrence', 'compound', 'Howard', 'PROPN', []],
 ['Howard', 'nsubjpass', 'announced', 'VERB', [Terrence]],
 ['was', 'auxpass', 'announced', 'VERB', []],
 ['announced', 'ROOT', 'announced', 'VERB', [Howard, was, in, in, .]],
 ['in', 'prep', 'announced', 'VERB', [role]],
 ['the', 'det', 'role', 'NOUN', []],
 ['role', 'pobj', 'in', 'ADP', [the, of]],
 ['of', 'prep', 'role', 'NOUN', [Rhodes]],
 ['Jim', 'compound', 'Rhodes', 'PROPN', []],
 ['Rhodes', 'pobj', 'of', 'ADP', [Jim]],
 ['in', 'prep', 'announced', 'VERB', [October]],
 ['October', 'pobj', 'in', 'ADP', [2006]],
 ['2006', 'nummod', 'October', 'PROPN', []],
 ['.', 'punct', 'announced', 'VERB', []]]

In [91]:
get_dependency_tree_nodes('Sebastian Stan makes an uncredited appearance in the post-credits scene reprising his role as Bucky Barnes.')

[['Sebastian', 'compound', 'Stan', 'PROPN', []],
 ['Stan', 'nsubj', 'makes', 'VERB', [Sebastian]],
 ['makes', 'ROOT', 'makes', 'VERB', [Stan, appearance, .]],
 ['an', 'det', 'appearance', 'NOUN', []],
 ['uncredited', 'amod', 'appearance', 'NOUN', []],
 ['appearance', 'dobj', 'makes', 'VERB', [an, uncredited, in]],
 ['in', 'prep', 'appearance', 'NOUN', [scene]],
 ['the', 'det', 'scene', 'NOUN', []],
 ['post', 'compound', 'credits', 'NOUN', []],
 ['-', 'punct', 'credits', 'NOUN', []],
 ['credits', 'compound', 'scene', 'NOUN', [post, -]],
 ['scene', 'pobj', 'in', 'ADP', [the, credits, reprising]],
 ['reprising', 'acl', 'scene', 'NOUN', [role]],
 ['his', 'poss', 'role', 'NOUN', []],
 ['role', 'dobj', 'reprising', 'VERB', [his, as]],
 ['as', 'prep', 'role', 'NOUN', [Barnes]],
 ['Bucky', 'compound', 'Barnes', 'PROPN', []],
 ['Barnes', 'pobj', 'as', 'ADP', [Bucky]],
 ['.', 'punct', 'makes', 'VERB', []]]

In [92]:
doc = nlp ('the role of Sonny Corleone was awarded to Carmine Caridi.')
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Sonny Corleone 12 26 PERSON
Carmine 42 49 ORG


In [93]:
get_dependency_tree_nodes('the role of Sonny Corleone was awarded to Carmine Caridi.')

[['the', 'det', 'role', 'NOUN', []],
 ['role', 'nsubjpass', 'awarded', 'VERB', [the, of]],
 ['of', 'prep', 'role', 'NOUN', [Corleone]],
 ['Sonny', 'compound', 'Corleone', 'PROPN', []],
 ['Corleone', 'pobj', 'of', 'ADP', [Sonny]],
 ['was', 'auxpass', 'awarded', 'VERB', []],
 ['awarded', 'ROOT', 'awarded', 'VERB', [role, was, to, .]],
 ['to', 'dative', 'awarded', 'VERB', [Caridi]],
 ['Carmine', 'compound', 'Caridi', 'PROPN', []],
 ['Caridi', 'pobj', 'to', 'ADP', [Carmine]],
 ['.', 'punct', 'awarded', 'VERB', []]]